In [1]:
import os
# import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from lightfm import LightFM
from lightfm.evaluation import precision_at_k
from scipy.sparse import coo_matrix

/Users/zhouzihui/anaconda3/lib/python3.7/site-packages/lightfm/_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn('LightFM was compiled without OpenMP support. '


In [2]:
NumberOfRows = 10000
filePath = 'Organized_dataset/cleaned/'
%matplotlib inline

https://github.com/lyst/lightfm

In [3]:
business = pd.read_csv(filePath + "business_cleaned.csv", nrows=10)
user = pd.read_csv(filePath + 'user_cleaned.csv', nrows=10)
ratings = pd.read_csv(filePath + 'review_cleaned_withTrainTest.csv', nrows=10)

In [4]:
business.head(1)

,address,attributes,business_id,categories,city,hours,is_open,latitude,longitude,name,postal_code,review_count,stars,state
0,30 Eglinton Avenue W,"{'RestaurantsReservations': 'True', 'GoodForMe...",QXAEGFB4oINsVuTFxEYKFQ,"Specialty Food, Restaurants, Dim Sum, Imported...",Mississauga,"{'Monday': '9:0-0:0', 'Tuesday': '9:0-0:0', 'W...",1,43.605499,-79.652289,Emerald Chinese Restaurant,L5R 3E7,128,2.5,ON


In [5]:
user.head(1)

,yelping_since,useful,compliment_photos,compliment_list,compliment_funny,compliment_plain,review_count,friends,fans,compliment_note,...,compliment_cute,average_stars,user_id,compliment_more,elite,compliment_hot,cool,name,compliment_profile,compliment_cool
0,2013-10-08 23:11:33,84,0,0,1,1,95,"c78V-rj8NQcQjOI8KP3UEA, alRMgPcngYSCJ5naFRBz5g...",5,1,...,0,4.03,l6BmjZMeQD3rDxWUbiAiow,0,"2015,2016,2017",2,25,Rashmi,0,1


In [6]:
ratings.head(1)

,user_id,date,business_id,cool,funny,review_id,stars,text,useful,TrainTest
0,9jdES117z1Dat4aJuGZs5w,2015-01-01 00:00:30,qEHYU_tm4YM04a0586UPvw,0.0,0.0,hNwleSHcvNksIQT31569Yg,5.0,Window washing inside and out done after many ...,5.0,1


In [7]:
businuessColumns = ['state', 'business_id'] #'review_count', 'stars', 
userColumns = ["user_id", "useful", "fans", "funny", "cool", "average_stars", "review_count"] #, "elite", "compliment_photos", "compliment_list", "compliment_funny", "compliment_plain", "compliment_note", "compliment_writer", "compliment_cute", "compliment_more", "compliment_hot", "compliment_profile", "compliment_cool",
ratingsColumns = ["business_id", "stars", "user_id"] #"cool", "funny", "useful", 
targetColumn = "stars"

In [8]:
business1 = pd.read_csv(filePath + "business_cleaned.csv", usecols=businuessColumns)
user1 = pd.read_csv(filePath + 'user_cleaned.csv', usecols=userColumns)
ratings1 = pd.read_csv(filePath + 'review_cleaned_withTrainTest.csv', usecols=ratingsColumns, nrows=NumberOfRows)

In [9]:
user1.head(1)

,useful,review_count,fans,funny,average_stars,user_id,cool
0,84,95,5,17,4.03,l6BmjZMeQD3rDxWUbiAiow,25


In [10]:
ratings1 = pd.merge(ratings1, business1, how='left', left_on='business_id', right_on='business_id')
ratings1 = pd.merge(ratings1, user1, how='left', left_on='user_id', right_on='user_id')

In [11]:
ratings1.head()

,user_id,business_id,stars,state,useful,review_count,fans,funny,average_stars,cool
0,9jdES117z1Dat4aJuGZs5w,qEHYU_tm4YM04a0586UPvw,5.0,NV,21,21,0,4,4.79,6
1,UaUVIQweBNlE_tVBCZjYdA,XcWlBj5oQgzKhR7Cxovj3w,2.0,AZ,34,30,2,4,3.80,8
2,mq5rKhLMHLbUaBeZY8mY8Q,O-uIEuv7JLUHajkemx_sVw,1.0,ON,8,19,0,6,2.68,2
3,-ELGAON2OCSBBIbGKNiYGQ,nqgeTj6bfIMY0v2J-vZa8A,5.0,AZ,1,9,0,0,3.33,0
4,tYxumQ3zkWje5X14LTDpcA,kd1NhNWvWo5AhBUSaGeSiw,5.0,ON,3,6,0,7,4.00,2


In [12]:
ratings1.shape, user1.shape, business1.shape

((10000, 10), (818922, 7), (158525, 2))

In [13]:
# Transform user ID and business ID to numbers
ratings1['business_id'] = ratings1['business_id'].astype('category')
ratings1['business_id'] = ratings1['business_id'].cat.codes

ratings1['user_id'] = ratings1['user_id'].astype('category')
ratings1['user_id'] = ratings1['user_id'].cat.codes

In [14]:
def convertDummies(df, column_name):
    df = pd.concat([
        df.drop(column_name, axis=1),
        pd.get_dummies(df[column_name], prefix=column_name+'_')
    ], axis=1)
    print(column_name, ' is ready.')
    return df

In [15]:
ratings1

,user_id,business_id,stars,state,useful,review_count,fans,funny,average_stars,cool
0,1109,5849,5.0,NV,21,21,0,4,4.79,6
1,3246,3694,2.0,AZ,34,30,2,4,3.80,8
2,5243,2664,1.0,ON,8,19,0,6,2.68,2
3,29,5565,5.0,AZ,1,9,0,0,3.33,0
4,5941,5235,5.0,ON,3,6,0,7,4.00,2
5,4494,2230,3.0,NV,24,10,4,15,3.90,16
6,4816,1163,5.0,AZ,4,16,0,3,3.71,2
7,4419,2318,5.0,AZ,8,14,0,2,3.67,0
8,2455,6722,2.0,NV,2,5,0,4,3.40,2
9,3290,4133,4.0,NV,153,135,1,32,3.78,32


In [16]:
ratings.head()

,user_id,date,business_id,cool,funny,review_id,stars,text,useful,TrainTest
0,9jdES117z1Dat4aJuGZs5w,2015-01-01 00:00:30,qEHYU_tm4YM04a0586UPvw,0.0,0.0,hNwleSHcvNksIQT31569Yg,5.0,Window washing inside and out done after many ...,5.0,1
1,UaUVIQweBNlE_tVBCZjYdA,2015-01-01 00:02:20,XcWlBj5oQgzKhR7Cxovj3w,0.0,0.0,DfZGAhAkPMJYDdXGRjhw8A,2.0,"I will admit, I do not have high expectations ...",0.0,0
2,mq5rKhLMHLbUaBeZY8mY8Q,2015-01-01 00:02:35,O-uIEuv7JLUHajkemx_sVw,0.0,1.0,65viXwIysYSxEyPZgRSVbQ,1.0,The chicken curry I got was extremely dry. Des...,0.0,1
3,-ELGAON2OCSBBIbGKNiYGQ,2015-01-01 00:02:42,nqgeTj6bfIMY0v2J-vZa8A,0.0,0.0,hS3phsfoP-fAZVlMomx4Kg,5.0,Really took care of me on my trip out of state...,0.0,0
4,tYxumQ3zkWje5X14LTDpcA,2015-01-01 00:04:13,kd1NhNWvWo5AhBUSaGeSiw,1.0,0.0,eaDgBBrOtvFUto5pADZwQA,5.0,Gary and Chester run this 2 year old Church St...,1.0,0


In [17]:
ratings = convertDummies(ratings1, 'business_id')
ratings = convertDummies(ratings, 'user_id')
ratings = convertDummies(ratings, 'state')

business_id  is ready.
user_id  is ready.
state  is ready.


In [18]:
ratings.head()

,stars,useful,review_count,fans,funny,average_stars,cool,business_id__0,business_id__1,business_id__2,...,state__AZ,state__IL,state__NC,state__NV,state__OH,state__ON,state__PA,state__QC,state__SC,state__WI
0,5.0,21,21,0,4,4.79,6,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1,2.0,34,30,2,4,3.80,8,0,0,0,...,1,0,0,0,0,0,0,0,0,0
2,1.0,8,19,0,6,2.68,2,0,0,0,...,0,0,0,0,0,1,0,0,0,0
3,5.0,1,9,0,0,3.33,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
4,5.0,3,6,0,7,4.00,2,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [19]:
train = ratings.iloc[:-2000]
test = ratings.iloc[-2000:]

In [20]:
train.shape, test.shape

((8000, 13572), (2000, 13572))

In [21]:
train.head()

,stars,useful,review_count,fans,funny,average_stars,cool,business_id__0,business_id__1,business_id__2,...,state__AZ,state__IL,state__NC,state__NV,state__OH,state__ON,state__PA,state__QC,state__SC,state__WI
0,5.0,21,21,0,4,4.79,6,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1,2.0,34,30,2,4,3.80,8,0,0,0,...,1,0,0,0,0,0,0,0,0,0
2,1.0,8,19,0,6,2.68,2,0,0,0,...,0,0,0,0,0,1,0,0,0,0
3,5.0,1,9,0,0,3.33,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
4,5.0,3,6,0,7,4.00,2,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [22]:
# train_pivot = train.pivot(index='user_id', columns='business_id', values='stars')
# train_pivot = train_pivot.fillna(0)
# train_pivot.head()

In [23]:
# test_pivot = test.pivot(index='user_id', columns='business_id', values='stars')
# test_pivot = test_pivot.fillna(0)
# test_pivot.head()

In [24]:
# from lightfm import LightFM
# from lightfm.datasets import fetch_movielens
# from lightfm.evaluation import precision_at_k

# Load the MovieLens 100k dataset. Only five
# star ratings are treated as positive.
# data = fetch_movielens(min_rating=5.0)

# Instantiate and train the model
# model = LightFM(loss='warp')
# model.fit(data['train'], epochs=30)

# # Evaluate the trained model
# test_precision = precision_at_k(model, data['test'], k=5).mean()

In [25]:
train_coo = coo_matrix(train)
test_coo = coo_matrix(test)

In [26]:
model = LightFM(learning_rate=0.05, loss='warp')
model.fit(train_coo, epochs=10)
train_precision = precision_at_k(model, train_coo, k=3).mean()

In [27]:
train_precision

0.99104166

In [28]:
model.predict(test_coo)

TypeError: predict() missing 1 required positional argument: 'item_ids'

In [ ]:
test_precision = precision_at_k(model, test_coo, k=3).mean()
test_precision

In [ ]:
test_coo

In [ ]:
from lightfm.evaluation import auc_score
train_auc = auc_score(model, train_coo).mean()
train_auc

In [ ]:
test_auc = auc_score(model, test_coo).mean()
test_auc

In [32]:
np.unique(ratings1['state'])

array(['AB', 'AZ', 'IL', 'NC', 'NV', 'OH', 'ON', 'PA', 'QC', 'SC', 'WI'],
      dtype=object)

In [36]:
ratings1['state']=='NY'

0       False
1       False
2       False
3       False
4       False
5       False
6       False
7       False
8       False
9       False
10      False
11      False
12      False
13      False
14      False
15      False
16      False
17      False
18      False
19      False
20      False
21      False
22      False
23      False
24      False
25      False
26      False
27      False
28      False
29      False
        ...  
9970    False
9971    False
9972    False
9973    False
9974    False
9975    False
9976    False
9977    False
9978    False
9979    False
9980    False
9981    False
9982    False
9983    False
9984    False
9985    False
9986    False
9987    False
9988    False
9989    False
9990    False
9991    False
9992    False
9993    False
9994    False
9995    False
9996    False
9997    False
9998    False
9999    False
Name: state, Length: 10000, dtype: bool